In [2]:
load('data/phonetrain.rdata')

In [5]:
train2 = list()
for(aclass in outclass) {
    train2[[aclass]] = traindata[[aclass]][1:500,]
    print(dim(train2[[aclass]]))
}

print(outclass)

[1] 500   3
[1] 500   3
[1] 500   3
[1] 500   3
[1] 500   3
[1] 500   3
[1] "Phoneonback"   "Phoneonbottom" "Phoneonfront"  "Phoneonleft"  
[5] "Phoneonright"  "Phoneontop"   


In [6]:

pgm_train <- function(outclass, train_data){
    result = list()
    for(class in outclass) {
        train = train_data[[class]] 
        mu1 = colMeans(train)
        sigma1 = cov(train, train)
        prec1 = solve(sigma1)
        detsig_log = log(det(sigma1))
        N1 =  nrow(train)
        l = list(mu1=mu1, sigma1=sigma1, prec1=prec1, detsig_log=detsig_log, N1=N1)
        result[[class]] = l
    }
    return(result)
}
result = pgm_train(outclass, train2)
result[1]

,x,y,z
x,6.382636e-04,-6.211061e-05,1.310078e-05
y,-6.211061e-05,6.812823e-04,-3.367878e-05
z,1.310078e-05,-3.367878e-05,1.001629e-03
,x,y,z
x,1581.02638,143.35381,-15.85886
y,143.35381,1483.26227,47.99823
z,-15.85886,47.99823,1000.19502


In [31]:
load('data/phonetest1.rdata')
library('mvtnorm')
model1 = pgm_train(outclass, traindata)

pgm_predict <- function(model, testds1_feature){
    if(ncol(testds1_feature) != 3){
        return(NULL)
    }
    vec = c()
    for(row_idx in 1:nrow(testds1_feature)){
        x = testds1_feature[row_idx,]
        x = as.matrix(x)
        p_max = -1
        target_idx = 1
        for(i in 1:length(model)){
            # model
            m = model[[i]]

            # mu, sigma
            sigma = m$sigma1
            mu = m$mu1
            sigma_inv = solve(sigma)

            p = 1
            p = det(sigma)** (-1/2) 
            v = exp((-1/2) * ((x - mu ) %*% sigma_inv  %*% t(x - mu)))
            p = p * v
            if(p > p_max){
                p_max = p
                target_idx = i
            }
            # library compute answer (cheating)
            # answer <- dmvnorm(x, m$mu1, m$sigma1)
            # print(answer)
        }
        vec[row_idx] = target_idx
    }
    return(vec)
}

vec = pgm_predict(model1, testds1_feature)
vec[1:50]

[1] 3 1 1 3 2 3 6 2 6 4 3 1 5 2 1 3 4 2 2 4 4 3 1 3 5 5 2 4 6 5 4 3 2 5 2 5 3 3
[39] 5 4 4 3 1 3 1 3 5 3 5 5

In [1]:
load(file="data/o_cost_train.rdata")
dm_train_t = as.numeric(ds4a_train[,1] == "pos")
tall=as.matrix(dm_train_t)
xmat = model.matrix(~f_past+g1+g2+g3+g4+g5+g6+g7+g8+g9+g10,
data=ds4a_train[,-1])


In [38]:
sigmoid <- function(x){
    return((exp(x) / (exp(x) + 1)))
}

logicreg_l2_train <- function(tall, xmat, debuglevel=0){
    lambda_rate =  0.0005
    param_tol = 10 ** (-5)
    granditertol = 2
    outitermax = 50
    inneritermax = 20
    N = nrow(tall)
    M = dim(xmat)[2]
    I = diag(M)
    # compute init lambda
    lambda = lambda_rate * N
    # compute init w
    w = solve(lambda * I + t(xmat) %*% (xmat) ) %*% t(xmat) %*% tall
    R = NULL
    for(outer_loop in 1:outitermax){
        w_count = 0
        for(inner_loop in 1:inneritermax){
            y = sigmoid(  t(w) %*% t(xmat))
            y = t(y)
            R = y %*% t(1-y) * diag(nrow(y)) 

            E = lambda * w + t(xmat) %*% ( y - tall)
            H = t(xmat) %*% R %*% xmat + lambda * I
            w_new = w - solve(H) %*% E
            w_diff = w_new - w
            mean_diff = mad(w_diff,center= mean(w_diff))
            w = w_new
            w_count = w_count + 1
            if(mean_diff < param_tol){
                # converge
                # print(w_new)
                break
            }
            
        }
        
        
        for(inner_loop in 1:inneritermax){
             ev = eigen(t(xmat) %*% R %*%  xmat)
            ev_values = ev$values
            gamma = 0
            for(i in 1:length(ev_values)){
                gamma  = gamma + (ev_values[i] / (ev_values[i] + lambda) )
            }
            lambda_new = gamma / (t(w) %*% w)
            lambda_diff = lambda_new - lambda
            lambda = drop(lambda_new)
            if(abs(lambda_diff) < param_tol){
                # converge
                # print(lambda)
                break
            }
        }
        
        if(w_count <= granditertol){
            break;
        }
        
    }
    y = sigmoid(  t(w) %*% t(xmat))
    y = t(y)
    R = y %*% t(1-y) * diag(nrow(y)) 
    Sn = solve(lambda * I + t(xmat) %*% R %*%  xmat)
    Sn = sqrt(diag(Sn))
    result = list(w = w, w_sd = Sn, lambda = lambda, M = M, N = N)
    return(result)
    
    
}

logicreg_l2_train(tall, xmat)


(Intercept),-1.4053791
f_past,-2.1991748
g1,0.8093526
g2,-8.5513319
g3,-6.9447606
g4,4.5614423
g5,0.3788263
g6,2.4862296
g7,-4.8258687
g8,-4.5343458
g9,-9.7827331


In [40]:
dm_train_t = as.numeric(ds4a_train[,1] == "pos")
tall=as.matrix(dm_train_t)
xmat = model.matrix(~f_past+g1+g2+g3+g4+g5+g6+g7+g8+g9+g10,data=ds4a_train[,-1])
model1 = logicreg_l2_train(tall, xmat, debuglevel=0)
#perform prediction
xmattest1 = model.matrix(~f_past+g1+g2+g3+g4+g5+g6+g7+g8+g9+g10,
data=ds4a_train[,-1])


In [41]:
model1

(Intercept),-1.4053791
f_past,-2.1991748
g1,0.8093526
g2,-8.5513319
g3,-6.9447606
g4,4.5614423
g5,0.3788263
g6,2.4862296
g7,-4.8258687
g8,-4.5343458
g9,-9.7827331


In [69]:
logicreg_l2_predict <- function(model1, xmattest1){
    w = (model1)$w
    prob = sigmoid( t(w) %*% t(xmattest1))
    class = apply(prob, 1, function(x) x > 0.5)
    class = as.integer(as.logical(class))
    prob = t(prob)
    result = list(prob = prob, class = class)
    return(result)
}
                  
result = logicreg_l2_predict(model1, xmattest1)
head(result$class,n=25)
head(result$prob[,1],n=25)

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

126       234       235       236       256       522       643       645 
0.2448322 0.2116120 0.2054513 0.1935176 0.2094563 0.1918210 0.1883066 0.1889833 
      662       712       739       776       894       897       898       899 
0.1975938 0.1893408 0.2117428 0.1858589 0.1851180 0.1813298 0.1780135 0.1673133 
     2360      2361      2397      2442      2457      2458      2510      2513 
0.1702156 0.2426719 0.1872902 0.2024119 0.1956855 0.1745073 0.2607178 0.1967121 
     2514 
0.1066816